# Finished Components

Write the code on personal jupyterhub, and test on there, when done, paste your function into a new cell on here, and create a cell underneath to show a test of your function that works.

In [1]:
# PUT ALL IMPORTS FOR YOUR FUNCTIONS HERE
import numpy as np
import scipy.stats as stats
import os
import pwd
import re

USER = pwd.getpwuid( os.getuid() )[ 0 ]

### processTagsTSV
- processes tags from inputted TSV file into a dictionary of dictionaries, with the first key being chromosome pointing to a dictionary with keys bp position and value number of reads at that position

In [2]:
# Global variable allCounts (dictionary of dictionaries), so that you do not have to keep running the cell
# after initial run to test code that relies on it.
def processTagTSVs(dir_name):
    allCounts = {}
    files = os.listdir(dir_name)
    for file_name in files:
        if file_name.endswith(".tags.tsv"):
            fileCounts = {}
            with open(os.path.join(dir_name, file_name)) as tsv:
                for line in tsv:
                    line = line.strip("\n")
                    line = line.split("\t")
                    for i in range(int(line[2]), int(line[2]) + int(line[-1])):
                        key = i
                        if key not in fileCounts.keys():
                            fileCounts[key] = int(float(line[4]))
                        else:
                            fileCounts.update({key : fileCounts[key]+int(float(line[4]))})
            fileCounts = {key : value for key, value in fileCounts.items() if value > 1}
            sortKeys = list(fileCounts.keys())
            sortKeys.sort()
            fileCounts = {i : fileCounts[i] for i in sortKeys}
            allCounts[file_name.split(".")[0]] = fileCounts
    
    sortKeys = list(allCounts.keys())
    sortKeys.sort()
    allCounts = {i : allCounts[i] for i in sortKeys}

    return allCounts

In [4]:
global allCounts
def testProcessTagsTSVs():

    allCounts = processTagTSVs("/home/%s/teams/25/tagdirs/Sox2"%USER)
    i = 0
    print(len(allCounts))
    for chrom, cr_dict in allCounts.items():
        i = 0
        for x, y in cr_dict.items():
            print(x,y)
            i+=1
            if i > 100:
                break
                
testProcessTagsTSVs()

26
7350025 2
7350026 2
7350027 2
7350028 2
7350029 2
7350030 2
7350031 2
7350032 2
7350033 2
7350034 2
7350035 2
7350036 2
7350037 2
7350038 2
7350039 2
7350040 2
7350041 2
7350042 2
7350043 2
7350044 2
7405175 2
7405176 2
7405177 2
7405178 2
7405179 2
7405180 2
7405181 2
7405182 2
7405183 2
7405184 2
7405185 2
7405186 2
7405187 2
7405188 2
7405189 2
7405190 2
7405191 2
7405192 2
7405193 2
7405194 2
7405195 2
7405196 2
7405197 2
7405198 2
7405199 2
7405200 2
7405201 2
7405202 2
7405203 2
7405204 2
7405205 2
7405206 2
7405207 2
7405208 2
7405209 2
7405210 2
7405211 2
60445860 2
60445861 3
60445862 3
60445863 3
60445864 3
60445865 3
60445866 3
60445867 3
60445868 3
60445869 3
60445870 3
60445871 3
60445872 3
60445873 3
60445874 3
60445875 3
60445876 3
60445877 3
60445878 3
60445879 3
60445880 3
60445881 3
60445882 3
60445883 3
60445884 3
60445885 3
60445886 3
60445887 3
60445888 3
60445889 3
60445890 2
60445891 2
60445892 2
60445893 2
60445894 2
60445895 2
60445896 2
60445897 2
60445898 

### PLACEHOLDER FUNCTION (REPLACE WITH WHATEVER IS NEXT)

In [4]:
def placeholder():
    return

In [ ]:
def testPlaceholder():
    return

## BED File output:

Will take in the input of a list of start, end positions for each peak. This will output a BED file in the format that each line is the start position, tab, end position, tab, p-value, tab.

In [ ]:
paired_list = [(3,78),(99,174),(201,276)]

def makeBED(paired_list):
    file = open("myfile.bed", "w")
    with file as f:
            for pair in paired_list:
                f.write(str(pair[0]) + "\t" + str(pair[1]) + "\t" + str(1) + "\n")

makeBED(paired_list)

## Calculate Fold Change:

In [6]:

def calcMaxFoldChange(windowStart, windowEnd, cr_dict, input_cr_dict):
    #maxVal = 0
    TFcounter = 0
    ControlCounter = 0
   # print (cr_dict)
    #print("\n djfd")
    #print(input_cr_dict)
    #exit()
    #in between startPos and startPos+windowSize, find the position with MAX value
    for i in range(windowStart, windowEnd):
        #check if it is the new max
        if i in cr_dict.keys():
            TFcounter += cr_dict[i]
        if i in input_cr_dict.keys():
            ControlCounter += input_cr_dict[i]
    print ("TF: "+ str(TFcounter))
    print ("CC: " + str(ControlCounter))
    return TFcounter/ControlCounter

In [8]:
def findPeaks(cr_dict,input_cr_dict, fcThreshold, windowSize):
    possiblePeaks = []
    count = 0
    for key, value in cr_dict.items():
        print(str(key) + "\t" + str(value))
        count+=1
        if count == 100:
            break
    for key, value in input_cr_dict.items():
        print(str(key) + "\t" + str(value))
        count+=1
        if count == 100:
            break
            
    #hunter code outline below:
    #do it per chromosome
    #given a window size, that is how much you check. incrementn i by window size
    #temporarily work only on chr17
    #start window is first position
    windowStart = next(iter(cr_dict))
    windowEnd = windowStart + windowSize
    for pos in cr_dict:

        
        #print ("start: " + str(windowStart))
        #print ("end: " + str(windowEnd))
 
        #print (pos)
        #run maxFoldChange

        maxFoldChange = calcMaxFoldChange(windowStart, windowEnd, cr_dict, input_cr_dict)
    
        #check if this maxFoldChange is above our threshold
        if (maxFoldChange >= fcThreshold):
            possiblePeaks.append((windowStart,windowEnd))
            print (maxFoldChange)
        #if position is still in the window, since we already ran it, continue
        if pos < windowEnd:
            continue
        else:
            #update window otherwise
            windowStart = pos
            windowEnd = pos + windowSize
    
    return possiblePeaks
       

In [9]:
def testPeaks():
    cr_dict_sox2 = {}
    cr_dict_input = {}
    #get sox2 stuff for testing
    Sox2Counts = processTagTSVs("/home/%s/teams/25/tagdirs/Sox2"%USER)
    for chrom, cr_dict in Sox2Counts.items():
        if chrom == "1":
            cr_dict_sox2 = cr_dict
            break
    #get input stuff for testing
    controlCounts = processTagTSVs("/home/%s/teams/25/tagdirs/input"%USER)
    for chrom, cr_dict in controlCounts.items():
        if chrom == "1":
            cr_dict_input = cr_dict
            break
            
    possiblePeaks = findPeaks(cr_dict_sox2, cr_dict_input, fcThreshold = 2, windowSize = 10)
    print (possiblePeaks)
testPeaks()

46341144	2
46341145	2
46341146	2
46341147	2
46341148	2
46341149	2
46341150	2
46341151	2
46341152	2
46341153	2
46341154	2
46341155	2
46341156	2
46341157	2
46341158	2
46341159	2
46341160	2
46341161	2
46341162	2
46341163	2
46341164	2
46341165	2
46341166	2
46341167	2
46341168	2
46341169	2
46341170	2


StopIteration: 

In [ ]:
#ONLY RUN THIS ONCE WHEN OPENING THE NOTEBOOK
def testProcessTagsTSVsInput():
    # CHANGE USER BEFORE RUNNING OR IT WONT WORK FOR YOU
    global inputCounts
    #chr: cr_dict
    inputCounts = processTagTSVs("/home/hbcheng/teams/25/tagdirs/input")
    i = 0
    print(len(inputCounts))
    for chrom, cr_dict in inputCounts.items():
        i = 0
        for x, y in cr_dict.items():
            print(x,y)
            i+=1
            if i > 100:
                break
testProcessTagsTSVsInput()